<a href="https://colab.research.google.com/github/Gaurav-822/TextGenerationInclinedToWhatsAppText/blob/main/TextGenerationInclinedToWhatsAppText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installing Neccessary Packages

In [ ]:
!pip install sentencepiece

##Importing Pre-trained Model google/flan-t5-base (~1GB)

In [ ]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = TFT5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

#Fine Tuning the Model
Fine tune the model using Whatsapp imported text after processing the imported text

Upload the whatsapp imported text and rename it to chat.txt then run this cell

In [ ]:
# Code to preprocess the imported text
import re

# Using Regex to Clean the date and time stamps
def clear_time_stamps(text):
  pattern = r"\b\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} [APap][Mm] - \b"
  return re.sub(pattern, "", text)

file_path = 'chat.txt'
txt = ""

# Open the file in read mode ('r')
with open(file_path, 'r') as file:
    for line in file:
        txt += clear_time_stamps(line)

print(txt)

file_path = 'cleaned_chat1.txt'

# Open the file in write mode ('w')
with open(file_path, 'w') as file:
  file.write(txt)

Change the name of "Gaurav" and "Anaysha" with character 1 and character 2 names in the loop of this cell first

In [ ]:
# Training the model with the prepared chats
train_data = [
    ("Hello Anaysha!", "Hi!"),
    # Adding our chats in this list as a form of tuple
]

# Adding elements to train_data
temp = []
ip = ""
op = ""
c = 1
with open("cleaned_chat1.txt", 'r') as file:
    for line in file:
      if len(temp) == 2:
        train_data.append(tuple(temp))
        temp = []
      elif "Gaurav" in line:  # change "Gaurav" with character 1
          ip += (line + ' ')
          c = 0
      elif "Anaysha" in line: # change "Anaysha" with character 2
          op += (line + ' ')
          c = 1
      else:
          if c == 0:
            ip += (line + ' ')
          else:
            op += (line + ' ')
      if ip != "" and op != "":
        temp = [ip, op]
        ip = ""
        op = ""
        train_data.append(tuple(temp))
        temp = []

# Tokenize and prepare the training data
train_encodings = tokenizer.prepare_seq2seq_batch(
    src_texts=[item[0] for item in train_data],
    tgt_texts=[item[1] for item in train_data],
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

# Extract input and output tensors
input_ids = train_encodings["input_ids"]
attention_mask = train_encodings["attention_mask"]
labels = train_encodings["labels"]

# Compile and train the model
optimizer = Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer)
model.fit(
    {"input_ids": input_ids, "attention_mask": attention_mask},
    {"labels": labels},
    epochs=1,
    batch_size=1
)

# Test the fine-tuned model
input_text = "Do you love me?"
input_ids = tokenizer(input_text, return_tensors="tf").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


###Generate text with the trained model
Results will be inclined to the training chats provided

In [ ]:
# Test the fine-tuned model
input_text = "I love you <3"
input_ids = tokenizer(input_text, return_tensors="tf").input_ids

outputs = model.generate(input_ids)
# print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))